# Learning Deep Learning
First try to use Keras
Based on

https://nlpforhackers.io/keras-intro

In [19]:
import pickle
import numpy as np
import spacy
from keras.models import Sequential
from keras.layers import Dense, Conv1D, MaxPooling1D, Flatten, Embedding, LSTM
from keras.layers import Dense
from keras.utils import np_utils
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.preprocessing import LabelEncoder
from keras.preprocessing.sequence import pad_sequences


In [20]:
VALIDATION_SIZE = 500
np.random.seed(666)

In [21]:

def encodeY(Y):
    '''create one-hot (dummies) for output, see also https://machinelearningmastery.com/multi-class-classification-tutorial-keras-deep-learning-library/
    encode class values as integers
    '''
    encoder = LabelEncoder()
    encoder.fit(Y)
    encoded_Y = encoder.transform(Y)
    dummy_y = np_utils.to_categorical(encoded_Y)
    return dummy_y

In [22]:
df = pickle.load(open('newsdataset/news_dataset_annotated.pkl', mode ='rb'))
# remove missing values
print(df['v9_major'].map(str.isdigit).value_counts())
df = df[df['v9_major'].map(str.isdigit)]
# remove niche categories
df = df[df['v9_major'].map(df.v9_major.value_counts()>150)]

X_train, X_test, y_train, y_test = train_test_split(df['text'], encodeY(df['v9_major'].map(int)), test_size = 0.2)
del df

True     11124
False     1435
Name: v9_major, dtype: int64


In [23]:
vectorizer = CountVectorizer(min_df=5, max_df=.9)
X_train_onehot = vectorizer.fit_transform(X_train)

In [24]:
# dit zou werken als y_train nog steeds een pandas-series was
# numberoflabels = len(y_train.unique())
# maar dat is het dankzij onze encodeY-functie niet meer.
# vandaar even een hele lelijke hack:
numberoflabels = len(set((str(e) for e in y_train.tolist())))
numberoflabels

19

In [25]:
model = Sequential()
model.add(Dense (units = 500, activation = 'relu', input_dim = len(vectorizer.get_feature_names())))
#model.add(Dense(units = 400, activation = 'sigmoid'))
#model.add(Dense(units = 300, activation = 'sigmoid'))
model.add(Dense(units = 200, activation = 'sigmoid'))
# model.add(Dense(units = 100, activation = 'sigmoid'))
model.add(Dense(units = numberoflabels, activation = 'sigmoid'))

In [26]:
model.compile(loss='categorical_crossentropy', optimizer = 'adam', metrics=['accuracy'])
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_7 (Dense)              (None, 500)               11062000  
_________________________________________________________________
dense_8 (Dense)              (None, 200)               100200    
_________________________________________________________________
dense_9 (Dense)              (None, 19)                3819      
Total params: 11,166,019
Trainable params: 11,166,019
Non-trainable params: 0
_________________________________________________________________


In [27]:
model.fit(X_train_onehot[:-VALIDATION_SIZE], y_train[:-VALIDATION_SIZE], 
          epochs=3, verbose=1,
          validation_data=(X_train_onehot[-VALIDATION_SIZE:], y_train[-VALIDATION_SIZE:]))

Train on 7712 samples, validate on 500 samples
Epoch 1/3
7712/7712 [==============================] - 30s 4ms/step - loss: 1.9146 - acc: 0.4452 - val_loss: 1.2964 - val_acc: 0.6280
Epoch 2/3
7712/7712 [==============================] - 30s 4ms/step - loss: 0.6350 - acc: 0.8318 - val_loss: 1.2407 - val_acc: 0.6200
Epoch 3/3
7712/7712 [==============================] - 30s 4ms/step - loss: 0.1760 - acc: 0.9619 - val_loss: 1.2764 - val_acc: 0.6480


In [28]:
scores = model.evaluate(vectorizer.transform(X_test), y_test, verbose = True)
print("Accuracy: {}".format(scores[1]))

2053/2053 [==============================] - 1s 454us/step
Accuracy: 0.5996103263807134


## Convolutional Network

In [29]:
word2idx = {word: idx for idx, word in enumerate(vectorizer.get_feature_names())}
tokenize = vectorizer.build_tokenizer()
preprocess = vectorizer.build_preprocessor()
 
def to_sequence(tokenizer, preprocessor, index, text):
    words = tokenizer(preprocessor(text))
    indexes = [index[word] for word in words if word in index]
    return indexes

X_train_sequences = [to_sequence(tokenize, preprocess, word2idx, x) for x in X_train] 

In [30]:
# Compute the max lenght of a text
MAX_SEQ_LENGHT = len(max(X_train_sequences, key=len))
print("MAX_SEQ_LENGHT=", MAX_SEQ_LENGHT)
 
N_FEATURES = len(vectorizer.get_feature_names())
X_train_sequences = pad_sequences(X_train_sequences, maxlen=MAX_SEQ_LENGHT, value=N_FEATURES)
print(X_train_sequences[0])
 

MAX_SEQ_LENGHT= 3197
[22123 22123 22123 ... 16975  7401 16977]


In [31]:
model = Sequential()
model.add(Embedding(len(vectorizer.get_feature_names()) + 1,
                    64,  # Embedding size
                    input_length=MAX_SEQ_LENGHT))
model.add(Conv1D(64, 5, activation='relu'))
model.add(MaxPooling1D(5))
model.add(Flatten())
model.add(Dense(units=64, activation='relu'))
model.add(Dense(units=numberoflabels, activation='sigmoid'))
 
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
print(model.summary())

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_3 (Embedding)      (None, 3197, 64)          1415936   
_________________________________________________________________
conv1d_2 (Conv1D)            (None, 3193, 64)          20544     
_________________________________________________________________
max_pooling1d_2 (MaxPooling1 (None, 638, 64)           0         
_________________________________________________________________
flatten_2 (Flatten)          (None, 40832)             0         
_________________________________________________________________
dense_10 (Dense)             (None, 64)                2613312   
_________________________________________________________________
dense_11 (Dense)             (None, 19)                1235      
Total params: 4,051,027
Trainable params: 4,051,027
Non-trainable params: 0
_________________________________________________________________


In [32]:
model.fit(X_train_sequences[:-VALIDATION_SIZE], y_train[:-VALIDATION_SIZE], 
          epochs=3, verbose=True,
          validation_data=(X_train_sequences[-VALIDATION_SIZE:], y_train[-VALIDATION_SIZE:]))
 

Train on 7712 samples, validate on 500 samples
Epoch 1/3
7712/7712 [==============================] - 77s 10ms/step - loss: 2.6940 - acc: 0.1162 - val_loss: 2.5939 - val_acc: 0.1640
Epoch 2/3
7712/7712 [==============================] - 73s 9ms/step - loss: 2.2788 - acc: 0.2308 - val_loss: 2.1810 - val_acc: 0.3180
Epoch 3/3
7712/7712 [==============================] - 80s 10ms/step - loss: 1.5129 - acc: 0.4881 - val_loss: 2.0064 - val_acc: 0.4180


In [33]:
X_test_sequences = [to_sequence(tokenize, preprocess, word2idx, x) for x in X_test]
X_test_sequences = pad_sequences(X_test_sequences, maxlen=MAX_SEQ_LENGHT, value=N_FEATURES)

In [34]:
scores = model.evaluate(X_test_sequences, y_test, verbose=1)
print("Accuracy:", scores[1]) 
 

2053/2053 [==============================] - 7s 3ms/step
Accuracy: 0.39746712130043954


# LSTM

In [35]:
model = Sequential()
model.add(Embedding(len(vectorizer.get_feature_names()) + 1,
                    64,  # Embedding size
                    input_length=MAX_SEQ_LENGHT))
model.add(LSTM(64))
model.add(Dense(units=numberoflabels, activation='sigmoid'))
 
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
print(model.summary())
 

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_4 (Embedding)      (None, 3197, 64)          1415936   
_________________________________________________________________
lstm_2 (LSTM)                (None, 64)                33024     
_________________________________________________________________
dense_12 (Dense)             (None, 19)                1235      
Total params: 1,450,195
Trainable params: 1,450,195
Non-trainable params: 0
_________________________________________________________________
None


In [36]:
model.fit(X_train_sequences[:-VALIDATION_SIZE], y_train[:-VALIDATION_SIZE], 
          epochs=2, batch_size=128, verbose=1, 
          validation_data=(X_train_sequences[-VALIDATION_SIZE:], y_train[-VALIDATION_SIZE:]))
 

Train on 7712 samples, validate on 500 samples
Epoch 1/2
7712/7712 [==============================] - 212s 28ms/step - loss: 2.7929 - acc: 0.1154 - val_loss: 2.7316 - val_acc: 0.0960
Epoch 2/2
7712/7712 [==============================] - 201s 26ms/step - loss: 2.6767 - acc: 0.1452 - val_loss: 2.6630 - val_acc: 0.1240


In [37]:
scores = model.evaluate(X_test_sequences, y_test, verbose=1)
print("Accuracy:", scores[1])
 

2053/2053 [==============================] - 22s 10ms/step
Accuracy: 0.17096931320745307


# pretrained embeddings

bla
bla